
# The notebook create an embeddings representation of the text rule and show it a interactive chart. 
## The embedding model is a decoder style LLM


The notebook requires a working ollama container to run. The noteboook use the ollama restAPI service to communicate in localhost.  
Refer to https://hub.docker.com/r/ollama/ollama

docker pull ollama/ollama
docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

to use:
docker exec -it ollama ollama run llama3.2:1b

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from utils_ollama import query_model_embed
from utils import umap_func,extract_projections,umap_2d_scatter_chart,create_folder

In [2]:
df=pd.read_csv("out/1_elaborated_dataset.csv")
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button)
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice)
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger
2556,9959,False,"[{'name': 'RSS Feed', 'module_name': 'feed', '...","When Adafruit releases a new product, blink!",Ktup6MV2-when-adafruit-releases-a-new-product-...,Ktup6MV2,10.0,"When Adafruit releases a new product, blink!",[{'id': '/triggers/feed.new_feed_item_matches'...,F

In [3]:
# other models can be used. 
model="llama3.2:1b" # llama 1 billion parameters instruction finetuned
model_embedding_name=model.replace(":",".")
model_embedding_name

'llama3.2.1b'

In [6]:
#just a test
query=["Where is Tuscany"]
result = query_model_embed(query, model=model)
print(type(result),len(result),result)#check the embedding dimension. Ok 2048

<class 'list'> 2048 [0.01484809, 0.056658503, -0.009276819, 0.004406065, 0.04729293, -0.0057539693, 0.015784992, 0.001405994, -0.0014096831, 0.033636075, 0.0022359737, 0.0110346265, -0.020369949, 0.015558717, -0.018323662, -0.0135936, 0.0028827828, 0.041581187, 0.04214728, -0.005419367, -0.022992708, 0.020887744, -0.0067884107, 0.0069974745, -0.0025702731, 0.00030606284, -0.066258535, 0.0447868, -0.0141323125, 0.0022427372, 0.0068242922, 0.0146769, 0.021042734, 0.036510754, 0.016409675, 0.007373631, 0.0021607562, -0.010157619, 0.035143506, -0.014495195, 0.013689309, -0.02394279, 0.015973428, 0.022270218, -0.04628358, -0.008758001, -0.012973799, 0.0122323865, 0.018768946, -0.0067113587, -0.0034379056, -0.034170628, 0.042090036, -0.024408026, -0.020492474, 0.01540188, -0.016091323, -0.028649054, -0.008267808, -0.00063861214, 0.023515508, 0.019292863, -0.02810743, -0.011845883, 0.023668833, 0.021699037, 0.006332897, 0.015112763, 0.018277526, 0.02542523, 0.029290214, -0.025114391, 0.013398

### create a new dataframe column that contains the embedding representation of the "name" attribute parameter

In [ ]:
df['embeddings'] = df.apply(
    lambda row: query_model_embed(
        input=row['name']
        
    ),
    axis=1
)


In [31]:
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[5.8760797e-06, 0.018332329, 0.032247532, -0.0..."
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[-0.01548752, 0.007120902, 0.032546464, -0.005..."
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[-0.007748441, 0.0021770685, 0.036084786, -0.0..."
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[-0.0013106825, 0.00499818, 0.053116668, -0.00..."
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[-0.013440807, -0.019121878, 0.019484065, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger,"[

In [ ]:
import numpy as np

embeddings = np.array(df["embeddings"].tolist())
embeddings

array([[ 5.8760797e-06,  1.8332329e-02,  3.2247532e-02, ...,
         8.6899180e-03, -1.5169558e-02, -2.2120630e-03],
       [-1.5487520e-02,  7.1209020e-03,  3.2546464e-02, ...,
        -2.2918832e-02, -4.1488796e-03,  2.4867827e-02],
       [-7.7484410e-03,  2.1770685e-03,  3.6084786e-02, ...,
        -1.1790700e-02, -6.7398650e-03,  1.2477863e-02],
       ...,
       [-1.6386720e-02, -1.9023608e-03,  5.3132620e-02, ...,
        -2.0504981e-02, -3.2578380e-02, -3.3031139e-03],
       [-1.8889135e-02,  1.3798380e-02,  3.4995700e-02, ...,
         4.3293600e-03, -1.0879731e-02,  5.3803160e-03],
       [-1.6756753e-02, -2.3842216e-02,  5.7848380e-02, ...,
         1.2718450e-03, -1.1283917e-02,  2.0400228e-02]])

## Create a visual representation 

In [69]:
#create a 2d umap projection
#https://umap-learn.readthedocs.io/en/latest/
projections_2d=umap_func(embeddings, n_components=2)
projections_2d

c:\Users\nicleonard\AppData\Local\anaconda3\envs\hiis\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


generating projections with UMAP took: 17.67 sec


array([[ 9.047457  ,  7.2720633 ],
       [ 9.108465  ,  0.34210742],
       [ 5.008309  ,  3.1097593 ],
       ...,
       [ 7.4868855 ,  2.1120296 ],
       [ 9.730382  ,  2.7073228 ],
       [10.300993  ,  4.841025  ]], dtype=float32)

In [70]:
first_projection, second_projection=extract_projections(projections_2d)

In [71]:
df["umap_2d_first"]=first_projection
df["umap_2d_second"]=second_projection

In [72]:
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[5.8760797e-06, 0.018332329, 0.032247532, -0.0...",9.047457,7.272063
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[-0.01548752, 0.007120902, 0.032546464, -0.005...",9.108465,0.342107
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[-0.007748441, 0.0021770685, 0.036084786, -0.0...",5.008309,3.109759
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[-0.0013106825, 0.00499818, 0.053116668, -0.00...",10.075050,5.822060
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[-0.013440807, -0.019121878, 0.019484065, -0.0...",8.652840,5.963976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/a

In [10]:
#create and save a interactive scatter chart (saved into the out_folder)
umap_2d_scatter_chart(df,model_embedding_name)

### example for llama3.2.1
<img src="figures/llama3.2.1_umap_2d.png" width=1000>

## Some comments
It is interesting to note that there are indeed clusters (texts grouped by meaning) but their class (the goal=the color) does not describe them very well.

From this point of view this model is the worst for our purpose of separating classes well. This will be reflected in the final classification results.
<img src="figures/llama3.2.1_umap_2d_comments.png" width=1000>

## Save the elaborated dataframe

In [ ]:
#va cambiata la rappresentazione degli embedding da [,] a [] per poter usare in notebook 04

In [84]:
df.to_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv", index=False)

In [19]:
# just a check
#note "embeddings" column has , as separator (has to be changed)
df=pd.read_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv")
df


,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[5.8760797e-06, 0.018332329, 0.032247532, -0.0...",9.047457,7.272063
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[-0.01548752, 0.007120902, 0.032546464, -0.005...",9.108465,0.342107
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[-0.007748441, 0.0021770685, 0.036084786, -0.0...",5.008309,3.109759
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[-0.0013106825, 0.00499818, 0.053116668, -0.00...",10.075050,5.822060
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[-0.013440807, -0.019121878, 0.019484065, -0.0...",8.652840,5.963976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/a

In [ ]:
# Convert the embedding column to the desired format (to be aligned to the other embedding outputs suitable to 03_notebook and 04_notebook)

df['embeddings'] = df['embeddings'].apply(lambda x: x.replace(","," "))
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),[5.8760797e-06 0.018332329 0.032247532 -0.0...,9.047457,7.272063
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,[-0.01548752 0.007120902 0.032546464 -0.005...,9.108465,0.342107
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,[-0.007748441 0.0021770685 0.036084786 -0.0...,5.008309,3.109759
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),[-0.0013106825 0.00499818 0.053116668 -0.00...,10.075050,5.822060
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,[-0.013440807 -0.019121878 0.019484065 -0.0...,8.652840,5.963976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_h

In [21]:
#save it again
df.to_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv", index=False)